In [5]:
import json

with open('tr_token_cosmos_mapping.json') as f:
    token_list = json.load(f)

In [6]:
len(token_list)

30158

In [3]:
from transformers import GPT2LMHeadModel

cosmos_model = GPT2LMHeadModel.from_pretrained("ytu-ce-cosmos/turkish-gpt2")
cosmos_model

/Users/alibayram/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
import torch

from transformers import Gemma2Model, Gemma2ForCausalLM

gemma_model = Gemma2Model.from_pretrained("google/gemma-2-2b-it")
gemma_model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gemma2Model(
  (embed_tokens): Embedding(256000, 2304, padding_idx=0)
  (layers): ModuleList(
    (0-25): 26 x Gemma2DecoderLayer(
      (self_attn): Gemma2Attention(
        (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
        (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
        (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        (rotary_emb): Gemma2RotaryEmbedding()
      )
      (mlp): Gemma2MLP(
        (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
        (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
        (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
        (act_fn): PytorchGELUTanh()
      )
      (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      (post_feedforward_layernorm): Gemma2RMSNorm((2304,),

In [10]:
import torch
# create an empty tensor to store the embeddings of the tokens shape (len(token_list), gemma_model.embed_tokens.weight.shape[1])
cosmos_embeddings = cosmos_model.transformer.wte.weight
embeddings = torch.zeros(len(token_list), cosmos_embeddings.shape[1])
embeddings.shape

NameError: name 'cosmos_model' is not defined

In [11]:
# create an empty tensor to store the embeddings of the tokens shape (len(token_list), gemma_model.embed_tokens.weight.shape[1])
gemma_embeddings = gemma_model.embed_tokens.weight
embeddings = torch.zeros(len(token_list), gemma_embeddings.shape[1])
embeddings.shape

torch.Size([30158, 2304])

In [12]:
token_list[0], cosmos_embeddings[8714]

NameError: name 'cosmos_embeddings' is not defined

In [7]:
cosmos_model.can_generate()

True

In [13]:
e1 = gemma_embeddings[7871]
e2 = gemma_embeddings[58714]

e1 = e1 + e2
average = e1 / 2
average

tensor([-0.0067, -0.0076,  0.0065,  ...,  0.0170, -0.0135,  0.0192],
       grad_fn=<DivBackward0>)

In [15]:
# for each token in the token_list, get the corresponding embedding from the gemma model and store it in the embeddings tensor
# if there is more than one token in the token_list that maps to the same index, average the embeddings

for token_map in token_list:
    index = token_map['tr_token_id']
    cosmos_token_ids = token_map['cosmos_token_ids']
    embedding = cosmos_embeddings[cosmos_token_ids[0]]
    sum_embedding = embedding
    for cosmos_token_id in cosmos_token_ids[1:]:
        embedding = embedding + cosmos_embeddings[cosmos_token_id]
    if len(cosmos_token_ids) > 1:
        embedding = embedding / len(cosmos_token_ids)        
    embeddings[index] = embedding

embeddings[0]

NameError: name 'cosmos_embeddings' is not defined

In [14]:
count_of_zero_embeddings = 0
for i in range(len(embeddings)):
    if torch.all(embeddings[i] == 0):
        count_of_zero_embeddings += 1
count_of_zero_embeddings

30158

In [12]:
# save the embeddings tensor to 3 different files
order = 0
for i in range(0, len(embeddings), 10100):
    torch.save(embeddings[i:i+10100], f'tr_cosmos_embeddings_{order}.pt')
    order += 1


In [25]:
gemma_model

Gemma2Model(
  (embed_tokens): Embedding(256000, 2304, padding_idx=0)
  (layers): ModuleList(
    (0-25): 26 x Gemma2DecoderLayer(
      (self_attn): Gemma2Attention(
        (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
        (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
        (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        (rotary_emb): Gemma2RotaryEmbedding()
      )
      (mlp): Gemma2MLP(
        (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
        (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
        (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
        (act_fn): PytorchGELUTanh()
      )
      (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      (post_feedforward_layernorm): Gemma2RMSNorm((2304,),

In [23]:
embeddings.shape, len(token_list)

(torch.Size([30158, 2304]), 30158)

In [17]:
# change gemma2_model.embed_tokens.weight to the embeddings tensor
gemma_model.embed_tokens.weight = torch.nn.Parameter(embeddings)


In [19]:
gemma_model.config.vocab_size = len(token_list)

In [42]:
gemma_model.save_pretrained("tr_gemma2_model")

In [43]:
gemma2_model = Gemma2ForCausalLM.from_pretrained("tr_gemma2_model")
gemma2_model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(30158, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps=

In [86]:
from transformers import AutoTokenizer

tr_tokenizer = AutoTokenizer.from_pretrained("alibayram/tr_tokenizer")
tr_tokenizer.is_fast

True

In [88]:
tr_tokenizer.vocab

{'Vazife': 23130,
 'Soydaş': 10179,
 'Idea': 21996,
 'isinden': 4822,
 'Sön': 16120,
 'ilçesi': 6547,
 'ing': 1502,
 'Demirle': 9012,
 'çam': 25674,
 'Abadi': 16870,
 'Eurov': 3582,
 'imren': 9547,
 'lanması': 4245,
 'amorf': 7701,
 'Mazlum': 21406,
 'Ayva': 22072,
 'Ε': 317,
 'üslup': 12884,
 'Traş': 26903,
 'Orhan': 6648,
 'abone': 25438,
 'Manuel': 25481,
 'Tavsiye': 12814,
 'galibiyet': 6794,
 'asistan': 18870,
 'kalker': 10059,
 'Kamu': 8032,
 'önemli': 2040,
 'mahmuz': 26575,
 'gevşek': 28937,
 'Aman': 22982,
 'Bezelye': 14278,
 'Doğaçlama': 29802,
 'ҡ': 446,
 'Levent': 13232,
 'esma': 19136,
 'Çapar': 22434,
 'Kaçamak': 22240,
 'hısım': 27112,
 'Pey': 13201,
 'eradikasyon': 26666,
 'tures': 6898,
 'Kayak': 15490,
 'Şekillen': 11255,
 'Kümülüs': 27612,
 'Zıt': 27371,
 'kazağı': 25353,
 'iliş': 6099,
 'sın': 6055,
 'Elvan': 30011,
 'düş': 1731,
 'Ortala': 19641,
 'başlangıç': 16210,
 'tab': 2380,
 'Marş': 12173,
 'kısrak': 25990,
 'Bedeni': 19351,
 'bekleme': 12946,
 '부': 1142,
 '

In [1]:
# Load model directly
from transformers import GemmaTokenizerFast

gemma2_tokenizer = GemmaTokenizerFast.from_pretrained("google/gemma-2-2b-it")
gemma2_tokenizer.is_fast

/Users/alibayram/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

In [2]:
prompt = "aa"

In [3]:
inputs = gemma2_tokenizer(prompt, return_tensors="pt")

In [8]:
gemma2_model.generation_config

NameError: name 'gemma2_model' is not defined

In [79]:
inputs

{'input_ids': tensor([[   2, 6499]]), 'attention_mask': tensor([[1, 1]])}

In [9]:
outputs = gemma_model.generate(**inputs)


TypeError: The current model class (Gemma2Model) is not compatible with `.generate()`, as it doesn't have a language model head. Classes that support generation often end in one of these names: ['ForCausalLM', 'ForConditionalGeneration', 'ForSpeechSeq2Seq', 'ForVision2Seq'].

In [83]:
gemma2_tokenizer.decode(outputs[0])

'<bos>aa<2mass><2mass><eos>'

In [92]:
import torch

device = torch.device("mps")
device

device(type='mps')

In [90]:
device.type

'mps'

In [97]:
%pip install torch torchvision torchaudio



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [96]:
%python -m pip install --upgrade pip

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).
